In [1]:
import numpy as np
import plotly.graph_objects as go
from typing import Dict

In [2]:
import fit_error_dampened_sine_wave

In [3]:
from fit_error_dampened_sine_wave.math_functions import damp_sine_wave


x_samp = np.linspace(0, 10, 1000)
y_samp = damp_sine_wave(x_samp, frequency=1, damping_rate=0.1, phase=0)


fig = go.Figure().update_layout(width=900)

fig.add_trace(go.Scatter(name="fun", mode="lines", x=x_samp, y=y_samp))
fig.update_layout(
    xaxis_title="time (s)",
    yaxis_title="Signal Amplitude (arb. u.)",
)
fig.show()

In [4]:
import numpy as np
import numpy.typing as npt
import plotly.graph_objects as go
from scipy.optimize import curve_fit
from fit_error_dampened_sine_wave.math_functions import damp_sine_wave

# 1. Generate clean data
x_samp: npt.NDArray[np.float64] = np.linspace(0, 10, 1000)
true_params = dict(frequency=1.0, damping_rate=0.1, phase=0.0)
y_clean: npt.NDArray[np.float64] = damp_sine_wave(x_samp, **true_params)

# 2. Add noise
rng = np.random.default_rng(seed=42)
noise: npt.NDArray[np.float64] = rng.normal(scale=0.05, size=x_samp.shape)
y_noisy = y_clean + noise


# 3. Define a wrapper for curve_fit
def wrapper(
    x: npt.NDArray[np.float64], frequency: float, damping_rate: float, phase: float
) -> npt.NDArray[np.float64]:
    return damp_sine_wave(
        x, frequency=frequency, damping_rate=damping_rate, phase=phase
    )


# 4. Fit data with perturbed initial guess
initial_guess = [0.8, 0.6, 0.2]  # intentionally inaccurate
popt, pcov = curve_fit(wrapper, x_samp, y_noisy, p0=initial_guess)
fitted_params = dict(zip(["frequency", "damping_rate", "phase"], popt))

# 5. Visualize
fig = go.Figure().update_layout(width=900)

fig.add_trace(go.Scatter(name="True Function", mode="lines", x=x_samp, y=y_clean))
fig.add_trace(
    go.Scatter(name="Noisy Data", mode="lines", x=x_samp, y=y_noisy, line=dict(width=1))
)
fig.add_trace(
    go.Scatter(
        name="Fitted Curve",
        mode="lines",
        x=x_samp,
        y=wrapper(x_samp, *popt),
        line=dict(dash="dash"),
    )
)

fig.update_layout(
    xaxis_title="time (s)",
    yaxis_title="Signal Amplitude (arb. u.)",
    title=f"Fitted Params: {fitted_params}",
)
fig.show()

In [5]:
# True model parameters
true_params: Dict[str, float] = dict(
    frequency=10.0, amplitude=1.0, damping_rate=0.1, phase=np.pi
)


np_rng = np.random.default_rng(42)
# 1. Generate clean data
x_samp: npt.NDArray[np.float64] = np.linspace(0, 10, 1000)
y_clean: npt.NDArray[np.float64] = damp_sine_wave(x_samp, **true_params)

# 2. Add noise
rng = np.random.default_rng(seed=42)
noise: npt.NDArray[np.float64] = np_rng.normal(scale=0.05, size=x_samp.shape)
y_noisy = y_clean + noise


# 3. Define a wrapper for curve_fit
# Wrapper for curve_fit
def wrapper(
    x: npt.NDArray[np.float64],
    frequency: float,
    amplitude: float,
    damping_rate: float,
    phase: float,
) -> npt.NDArray[np.float64]:
    return damp_sine_wave(
        x,
        frequency=frequency,
        damping_rate=damping_rate,
        phase=phase,
        amplitude=amplitude,
    )


key_order = ("frequency", "amplitude", "damping_rate", "phase")

# 4. Fit data with perturbed initial guess
initial_guess = [true_params[k] for k in key_order]  # intentionally inaccurate
initial_guess = np.array(initial_guess)
initial_guess = (
    initial_guess + np_rng.normal(scale=0.9, size=initial_guess.shape) * initial_guess
)  # add some noise to the initial guess
print(initial_guess)
popt, pcov = curve_fit(wrapper, x_samp, y_noisy, p0=initial_guess)
fitted_params = dict(zip(["frequency", "damping_rate", "phase"], popt))

# 5. Visualize
fig = go.Figure().update_layout(width=900)

fig.add_trace(go.Scatter(name="True Function", mode="lines", x=x_samp, y=y_clean))
fig.add_trace(
    go.Scatter(
        name="Noisy Data", mode="markers", x=x_samp, y=y_noisy, line=dict(width=1)
    )
)
fig.add_trace(
    go.Scatter(
        name="Fitted Curve",
        mode="lines",
        x=x_samp,
        y=wrapper(x_samp, *popt),
        line=dict(dash="dash"),
    )
)

fig.update_layout(
    xaxis_title="time (s)",
    yaxis_title="Signal Amplitude (arb. u.)",
    title=f"Fitted Params: {fitted_params}",
)
fig.show()

[9.46645619 0.34364176 0.06269742 4.93393202]


In [6]:
10**4 * 10 * 2

200000

In [9]:
import fit_error_dampened_sine_wave.math_functions
import numpy as np
import numpy.typing as npt
import plotly.graph_objects as go
from scipy.optimize import curve_fit

from fit_error_dampened_sine_wave.math_functions import damp_sine_wave
from plotly.subplots import make_subplots
from tqdm import tqdm
from typing import Tuple
from multiprocessing import Pool, cpu_count

from typing import Tuple

np_rng = np.random.default_rng(42)


# Wrapper for curve_fit
def wrapper(
    x: npt.NDArray[np.float64],
    frequency: float,
    amplitude: float,
    damping_rate: float,
    phase: float,
) -> npt.NDArray[np.float64]:
    return damp_sine_wave(
        x,
        frequency=frequency,
        damping_rate=damping_rate,
        phase=phase,
        amplitude=amplitude,
    )


key_order = ("frequency", "amplitude", "damping_rate", "phase")

# True model parameters
true_params: Dict[str, float] = dict(
    frequency=1.0, amplitude=1.0, damping_rate=0.01, phase=np.pi / 2
)

# Settings
noise_scale = 0.01
num_repeats: int = 300
num_points: int = 1000  # points that the curve is sampling
# durations: npt.NDArray[np.float64] = np.linspace(0.1, 100, 100)
durations = np.logspace(-1.0, 4, 300)


# the problem is that the sampling can alias the true frequency, so we pick durations that do not alias to a frequency that is close to the niquist or zero frequency
# Step 1: Compute sampling frequencies
f_s_vals = num_points / durations
# Step 2: Compute apparent frequencies
f_a_vals = np.array(
    [
        fit_error_dampened_sine_wave.math_functions.apparent_frequency(
            true_params["frequency"], f_s
        )
        for f_s in f_s_vals
    ]
)
# Step 3: Normalize apparent frequencies by f_true
f_a_norm = f_a_vals / true_params["frequency"]
f_N_norm = f_s_vals / (2 * true_params["frequency"])
# Step 4: Mask out undesired edge cases
min_thresh = 0.05
max_thresh = 0.95
valid: npt.NDArray[np.bool_] = (f_a_norm > min_thresh) & (f_a_norm < max_thresh)

# Step 5: Adjust durations for invalid cases by nudging a fraction of the Nyquist zone
delta: float = 0.1  # fraction of Nyquist zone width
adjusted_durations: npt.NDArray[np.float64] = durations.copy()

for idx in np.where(~valid)[0]:
    f_s = f_s_vals[idx]
    N = int(np.floor(2 * true_params["frequency"] / f_s)) + 1
    f_s_zone_width = 2 * true_params["frequency"] * (1 / N - 1 / (N + 1))
    f_s_offset = delta * f_s_zone_width
    direction = 1 if N % 2 == 0 else -1
    f_s_adjusted = f_s + direction * f_s_offset
    adjusted_durations[idx] = num_points / f_s_adjusted

# Final assignment
durations = adjusted_durations


# Storage
rmse_results = {k: [] for k in true_params}
stderr_results = {k: [] for k in true_params}


def fit_for_duration(
    duration: float,
) -> Tuple[float, Dict[str, float], Dict[str, float]]:
    x_vals = np.linspace(0, duration, num_points)

    param_diffs = {k: [] for k in true_params}
    param_stderr = {k: [] for k in true_params}

    for _ in range(num_repeats):
        y_clean = damp_sine_wave(x_vals, **true_params)
        noise = np_rng.normal(scale=noise_scale, size=y_clean.shape)
        y_noisy = y_clean + noise

        try:
            initial_guess = np.array([true_params[k] for k in key_order])
            initial_guess += (
                np_rng.normal(scale=0.00001, size=initial_guess.shape) * initial_guess
            )
            popt, pcov = curve_fit(wrapper, x_vals, y_noisy, p0=initial_guess)
            perr = np.sqrt(np.diag(pcov))

            for i, key in enumerate(true_params):
                param_diffs[key].append(popt[i] - true_params[key])
                param_stderr[key].append(perr[i])
        except RuntimeError:
            for key in true_params:
                param_diffs[key].append(np.nan)
                param_stderr[key].append(np.nan)

    rmse = {
        key: np.sqrt(np.nanmean(np.square(param_diffs[key]))) for key in true_params
    }
    stderr = {key: np.nanmean(param_stderr[key]) for key in true_params}

    return duration, rmse, stderr


print("Starting parallel fits...")
with Pool(processes=cpu_count()) as pool:
    results = list(tqdm(pool.imap(fit_for_duration, durations), total=len(durations)))

rmse_results = {k: [] for k in true_params}
stderr_results = {k: [] for k in true_params}

# Extract from results
durations_out = []
for duration, rmse, stderr in results:
    durations_out.append(duration)
    for key in true_params:
        rmse_results[key].append(rmse[key])
        stderr_results[key].append(stderr[key])

durations = np.array(durations_out)

# Plotting with Plotly
# Create subplots
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01)

for i, k in enumerate(true_params, start=1):
    fig.add_trace(
        go.Scatter(
            x=durations, y=rmse_results[k], mode="lines+markers", name=f"RMSE {k}"
        ),
        row=i,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=durations,
            y=stderr_results[k],
            mode="lines+markers",
            name=f"stderr {k}",
            line=dict(dash="dot"),
        ),
        row=i,
        col=1,
    )

    xref = "x domain" if i == 1 else f"x{i} domain"
    yref = "y domain" if i == 1 else f"y{i} domain"
    fig.add_annotation(
        text=f"<b>{k}</b>",
        x=0.5,
        y=0.95,
        xref=xref,
        yref=yref,
        showarrow=False,
        font=dict(size=14),
        align="left",
    )

fig.update_layout(
    title="Fit RMSE and stderr vs Sample Duration (each parameter in separate subplot)",
    xaxis4_title="Sample Duration (s)",
    height=900,
    width=900,
    showlegend=True,
)

for i in range(1, 4):
    fig.update_yaxes(type="log", row=i, col=1)
    fig.update_xaxes(type="log", row=i, col=1)


fig.update_xaxes(type="log", title="Sample Duration (s)", row=4, col=1)

fig.show()

Starting parallel fits...


100%|██████████| 300/300 [00:10<00:00, 29.40it/s]


In [ ]:
true_params["phase"]

1.5707963267948966

In [13]:
import fit_error_dampened_sine_wave.anal_fit_err as anal_fit_err
import importlib
import dataclasses

importlib.reload(anal_fit_err)

# Generate theoretical predictions
theory_undamped_curves = {
    "amplitude": [],
    "frequency": [],
    "phase": [],
    "damping_rate": [],
}

theory_curves = {"amplitude": [], "frequency": [], "phase": [], "damping_rate": []}

theory_short_time_curves = {
    "amplitude": [],
    "frequency": [],
    "phase": [],
    "damping_rate": [],
}

for this_duration in durations:
    est = anal_fit_err.analy_err_in_fit_damp_sine(
        amplitude=true_params["amplitude"],
        samp_num=num_points,
        samp_time=this_duration,
        sigma_obs=noise_scale,
        damp_rate=true_params["damping_rate"],
    )
    theory_curves["amplitude"].append(est.amplitude)
    theory_curves["frequency"].append(est.frequency)
    theory_curves["phase"].append(est.phase)
    theory_curves["damping_rate"].append(est.damping_rate)

    est = anal_fit_err.analy_err_in_fit_cw_sine(
        amplitude=true_params["amplitude"],
        samp_num=num_points,
        samp_time=this_duration,
        sigma_obs=noise_scale,
    )
    theory_undamped_curves["amplitude"].append(est.amplitude)
    theory_undamped_curves["frequency"].append(est.frequency)
    theory_undamped_curves["phase"].append(est.phase)
    theory_undamped_curves["damping_rate"].append(np.nan)


for key in true_params:
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=durations, y=rmse_results[key], mode="lines+markers", name=f"RMSE {key}"
        )
    )
    fig.add_trace(
        go.Scatter(
            x=durations,
            y=stderr_results[key],
            mode="lines+markers",
            name=f"stderr {key}",
            line=dict(dash="dot"),
        )
    )
    fig.add_trace(
        go.Scatter(
            x=durations,
            y=theory_curves[key],
            mode="lines",
            name=f"theory {key}",
            line=dict(dash="dash"),
        )
    )

    fig.add_trace(
        go.Scatter(
            x=durations,
            y=theory_short_time_curves[key],
            mode="lines",
            name=f"theory short time {key}",
            line=dict(dash="dashdot"),
        )
    )

    fig.add_trace(
        go.Scatter(
            x=durations,
            y=theory_undamped_curves[key],
            mode="lines",
            name=f"theory undamped {key}",
            line=dict(dash="dash"),
        )
    )

    fig.update_layout(
        title=f"Comparison for {key}",
        xaxis_title="Sample Duration (s)",
        yaxis_title="Uncertainty",
        width=900,
        height=500,
        yaxis_type="log",
        xaxis_type="log",
    )
    fig.show()

In [14]:
import plotly.graph_objects as go
import plotly.io as pio

# Set default font to Computer Modern (may require CMU Serif if CM not available)
latex_font = "CMU Serif"
pio.templates["cm10_latex"] = go.layout.Template(
    layout=go.Layout(
        font=dict(family=latex_font, size=18),
        title_font=dict(family=latex_font, size=22),
        legend=dict(
            font=dict(family=latex_font, size=16),
            bordercolor="black",
            borderwidth=1,
        ),
        xaxis=dict(
            title_font=dict(family=latex_font, size=20),
            tickfont=dict(family=latex_font, size=16),
        ),
        yaxis=dict(
            title_font=dict(family=latex_font, size=20),
            tickfont=dict(family=latex_font, size=16),
        ),
        width=900,
        height=500,
        margin=dict(l=80, r=20, t=50, b=80),
    )
)
pio.templates.default = "cm10_latex"

In [15]:
def add_minor_log_ticks(
    fig: go.Figure, axis: str, range_vals: tuple[float, float], length: float = 0.01
):
    import numpy as np

    minor_ticks = []
    for decade in range(
        int(np.floor(np.log10(range_vals[0]))), int(np.ceil(np.log10(range_vals[1])))
    ):
        base = 10**decade
        for i in range(2, 10):  # 2 through 9
            tick = i * base
            if range_vals[0] <= tick <= range_vals[1]:
                minor_ticks.append(tick)

    for t in minor_ticks:
        if axis == "x":
            fig.add_shape(
                type="line",
                x0=t,
                x1=t,
                y0=0,
                y1=length,
                xref="x",
                yref="paper",
                line=dict(color="black", width=1),
                layer="above",
            )
        elif axis == "y":
            fig.add_shape(
                type="line",
                x0=0,
                x1=length,
                y0=t,
                y1=t,
                xref="paper",
                yref="y",
                line=dict(color="black", width=1),
                layer="above",
            )

In [16]:
true_params

{'frequency': 1.0,
 'amplitude': 1.0,
 'damping_rate': 0.01,
 'phase': 1.5707963267948966}

In [18]:
import os
import fit_error_dampened_sine_wave.str_utils as str_utils

# the chrome i installed didnt work
# os.environ["CHROME_PATH"] = "/usr/bin/google-chrome"  # or whatever `which google-chrome` gave
import kaleido

# chrome_path = "/usr/bin/google-chrome"
# kaleido.get_chrome_sync(chrome_path)

import numpy as np

x_min = min(durations)
x_max = max(durations)

y_min_rounded = np.floor(np.log10(x_min))
y_max_rounded = np.ceil(np.log10(x_max))
# Define major tick positions
x_range_log = [np.log10(x_min), np.log10(x_max)]
x_major_ticks = np.arange(y_min_rounded, y_max_rounded + 1, 1)
x_tick_labels = [str_utils.make_plotly_power_ten(int(v)) for v in x_major_ticks]


for key in true_params:
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=durations,
            y=rmse_results[key],
            mode="lines",
            name="Simulation",
            line=dict(color="black", width=2),
            opacity=0.8,
        )
    )

    # fig.add_trace(go.Scatter(
    #     x=durations, y=stderr_results[key],
    #     mode="lines+markers",
    #     name="stderr",
    #     line=dict(dash="dot", color="dimgray"),
    #     marker=dict(size=6, symbol="circle-open")
    # ))

    fig.add_trace(
        go.Scatter(
            x=durations,
            y=theory_curves[key],
            mode="lines",
            name="Theory Damped",
            line=dict(dash="dash", color="blue", width=2),
        )
    )

    if key != "damping_rate":
        fig.add_trace(
            go.Scatter(
                x=durations,
                y=theory_undamped_curves[key],
                mode="lines",
                name="Theory CW",
                line=dict(dash="dot", color="red", width=2),
            )
        )

    osc_period = 1 / true_params["frequency"]
    damping_time = 1 / true_params["damping_rate"]
    for vline in [osc_period, damping_time]:
        fig.add_shape(
            type="line",
            x0=vline,
            x1=vline,
            y0=min(rmse_results[key]) * 0.01,
            y1=max(rmse_results[key]) * 100,
            line=dict(color="limegreen", dash="dot", width=2),
            layer="below",
        )

    y_max = max(rmse_results[key])
    y_min = min(rmse_results[key])
    y_max_rounded = np.ceil(np.log10(y_max))
    y_min_rounded = np.floor(np.log10(y_min))
    if key == "frequency":
        y_min_rounded = y_min_rounded - 1
    y_range_log = [y_min_rounded, y_max_rounded]
    num_decades = y_max_rounded - y_min_rounded
    y_major_ticks = np.arange(y_min_rounded, y_max_rounded + 1, 1)
    print(y_major_ticks)

    y_tick_labels = [str_utils.make_plotly_power_ten(int(v)) for v in y_major_ticks]

    key_to_title = {
        "frequency": "Frequency Uncertainty (Hz)",
        "amplitude": "Amplitude Uncertainty (m)",
        "damping_rate": "Damping Rate Uncertainty (1/s)",
        "phase": "Phase Uncertainty  (rad)",
    }
    fig.update_layout(
        xaxis=dict(
            range=x_range_log,
            type="log",
            tickvals=10**x_major_ticks,
            ticktext=x_tick_labels,
            title="Sample Duration (s)",
            ticks="outside",
            showline=True,
            mirror=True,
            title_standoff=0,  # default is ~15–20, decrease to move label closer
        ),
        yaxis=dict(
            range=y_range_log,
            type="log",
            tickvals=10**y_major_ticks,
            ticktext=y_tick_labels,
            title=key_to_title[key],
            ticks="outside",
            showline=True,
            mirror=True,
            title_standoff=4,  # default is ~15–20, decrease to move label closer
        ),
        font=dict(family="CMU Serif", size=18),
        legend=dict(
            x=0.4,
            y=0.8,
            xanchor="center",
            yanchor="middle",
            bgcolor="rgba(255,255,255,1.0)",  # semi-transparent white background,
            font=dict(size=16),
            bordercolor="black",
            borderwidth=1,
        ),
        width=800,
        height=500,
        margin=dict(l=80, r=20, t=40, b=80),
    )

    # Add minor ticks
    # add_minor_log_ticks(fig, "x", (min(durations), max(durations)))
    # add_minor_log_ticks(fig, "y", (min(y_major_ticks), max(y_major_ticks)))

    fig.show()

    # Create export directory
    EXPORT_DIR = "figures"
    os.makedirs(EXPORT_DIR, exist_ok=True)

    # Define filename stem
    filename_base = os.path.join(EXPORT_DIR, f"uncertainty_{key}")

    # Export to SVG and PDF
    # pio.write_image(fig, f"{filename_base}.svg")
    pio.write_image(fig, f"{filename_base}.pdf")
    pio.write_image(fig, f"{filename_base}.png")

[-7. -6. -5. -4. -3. -2. -1.]


[-4. -3. -2. -1.]


[-5. -4. -3. -2. -1.  0.  1.]


[-4. -3. -2. -1.  0.]


# Multi Phase Results

Make Plots similar to above but show the simulation results for a range of different phases. This should show that the
results become phase sensitive at a small number of cycles.

In [ ]:
# Multi-phase results: Run fitting analysis for different initial phases
# and plot the same style plots as above, with 3 phases: 0, π/2, π

import os
import str_utils
import kaleido
import anal_fit_err
import importlib

importlib.reload(anal_fit_err)

# Define phases to analyze
phases_to_analyze = [0, np.pi / 4, np.pi / 2, np.pi * 3 / 4, np.pi]
phase_labels = ["φ = 0", "φ = π/4", "φ = π/2", "φ = 3π/4", "φ = π"]
phase_colors = ["black", "blue", "red", "green", "purple"]

# Storage for results across different phases
multi_phase_results = {}

# Run fitting for each phase
for phase_idx, test_phase in enumerate(phases_to_analyze):
    print(f"Running fits for phase {phase_labels[phase_idx]}...")

    rmse_results_phase = {k: [] for k in true_params}
    stderr_results_phase = {k: [] for k in true_params}

    def fit_for_duration_with_phase(
        duration: float,
    ) -> Tuple[float, Dict[str, float], Dict[str, float]]:
        x_vals = np.linspace(0, duration, num_points)
        param_diffs = {k: [] for k in true_params}
        param_stderr = {k: [] for k in true_params}

        for _ in range(num_repeats):
            # Create test params with this phase
            test_params = true_params.copy()
            test_params["phase"] = test_phase

            y_clean = damp_sine_wave(x_vals, **test_params)
            noise = np_rng.normal(scale=noise_scale, size=y_clean.shape)
            y_noisy = y_clean + noise

            try:
                initial_guess = np.array([true_params[k] for k in key_order])
                initial_guess += (
                    np_rng.normal(scale=0.00001, size=initial_guess.shape)
                    * initial_guess
                )
                popt, pcov = curve_fit(wrapper, x_vals, y_noisy, p0=initial_guess)
                perr = np.sqrt(np.diag(pcov))

                for i, key in enumerate(true_params):
                    param_diffs[key].append(popt[i] - test_params[key])
                    param_stderr[key].append(perr[i])
            except RuntimeError:
                for key in true_params:
                    param_diffs[key].append(np.nan)
                    param_stderr[key].append(np.nan)

        rmse = {
            key: np.sqrt(np.nanmean(np.square(param_diffs[key]))) for key in true_params
        }
        stderr = {key: np.nanmean(param_stderr[key]) for key in true_params}

        return duration, rmse, stderr

    print("Starting parallel fits...")
    with Pool(processes=cpu_count()) as pool:
        results = list(
            tqdm(
                pool.imap(fit_for_duration_with_phase, durations), total=len(durations)
            )
        )

    # Extract from results
    durations_out = []
    for duration, rmse, stderr in results:
        durations_out.append(duration)
        for key in true_params:
            rmse_results_phase[key].append(rmse[key])
            stderr_results_phase[key].append(stderr[key])

    multi_phase_results[phase_idx] = {
        "label": phase_labels[phase_idx],
        "color": phase_colors[phase_idx],
        "rmse": rmse_results_phase,
    }

# Compute theory curves once (independent of phase)
theory_undamped_curves = {
    "amplitude": [],
    "frequency": [],
    "phase": [],
    "damping_rate": [],
}
theory_curves = {"amplitude": [], "frequency": [], "phase": [], "damping_rate": []}

for this_duration in durations:
    est = anal_fit_err.analy_err_in_fit_damp_sine(
        amplitude=true_params["amplitude"],
        samp_num=num_points,
        samp_time=this_duration,
        sigma_obs=noise_scale,
        damp_rate=true_params["damping_rate"],
    )
    theory_curves["amplitude"].append(est.amplitude)
    theory_curves["frequency"].append(est.frequency)
    theory_curves["phase"].append(est.phase)
    theory_curves["damping_rate"].append(est.damping_rate)

    est = anal_fit_err.analy_err_in_fit_cw_sine(
        amplitude=true_params["amplitude"],
        samp_num=num_points,
        samp_time=this_duration,
        sigma_obs=noise_scale,
    )
    theory_undamped_curves["amplitude"].append(est.amplitude)
    theory_undamped_curves["frequency"].append(est.frequency)
    theory_undamped_curves["phase"].append(est.phase)
    theory_undamped_curves["damping_rate"].append(np.nan)

# Now create plots similar to the previous cell, but with 3 phases overlaid
x_min = min(durations)
x_max = max(durations)

y_min_rounded = np.floor(np.log10(x_min))
y_max_rounded = np.ceil(np.log10(x_max))
x_range_log = [np.log10(x_min), np.log10(x_max)]
x_major_ticks = np.arange(y_min_rounded, y_max_rounded + 1, 1)
x_tick_labels = [str_utils.make_plotly_power_ten(int(v)) for v in x_major_ticks]

for key in true_params:
    fig = go.Figure()

    # Add simulation traces for each phase
    for phase_idx in range(len(phases_to_analyze)):
        fig.add_trace(
            go.Scatter(
                x=durations,
                y=multi_phase_results[phase_idx]["rmse"][key],
                mode="lines",
                name=f"{multi_phase_results[phase_idx]['label']}",
                line=dict(color=multi_phase_results[phase_idx]["color"], width=2),
                opacity=0.8,
            )
        )

    # Add single theory curve
    fig.add_trace(
        go.Scatter(
            x=durations,
            y=theory_curves[key],
            mode="lines",
            name="Theory Damped",
            line=dict(color="blue", width=2, dash="dash"),
            opacity=0.6,
        )
    )

    # Add single undamped theory curve (except for damping_rate)
    if key != "damping_rate":
        fig.add_trace(
            go.Scatter(
                x=durations,
                y=theory_undamped_curves[key],
                mode="lines",
                name="Theory CW",
                line=dict(color="red", width=2, dash="dot"),
                opacity=0.4,
            )
        )

    # Add reference lines for period and damping time
    osc_period = 1 / true_params["frequency"]
    damping_time = 1 / true_params["damping_rate"]
    for vline in [osc_period, damping_time]:
        fig.add_shape(
            type="line",
            x0=vline,
            x1=vline,
            y0=1e-6,
            y1=1e2,
            line=dict(color="limegreen", dash="dot", width=2),
            layer="below",
        )

    # Calculate y-axis range
    all_rmse_values = []
    for phase_idx in range(len(phases_to_analyze)):
        all_rmse_values.extend(multi_phase_results[phase_idx]["rmse"][key])
    all_rmse_values.extend(theory_curves[key])
    all_rmse_values.extend(theory_undamped_curves[key])

    all_rmse_values = [v for v in all_rmse_values if not np.isnan(v)]
    y_max = max(all_rmse_values)
    y_min = min(all_rmse_values)
    y_max_rounded = np.ceil(np.log10(y_max))
    y_min_rounded = np.floor(np.log10(y_min))
    if key == "frequency":
        y_min_rounded = y_min_rounded - 1
    y_range_log = [y_min_rounded, y_max_rounded]
    y_major_ticks = np.arange(y_min_rounded, y_max_rounded + 1, 1)
    y_tick_labels = [str_utils.make_plotly_power_ten(int(v)) for v in y_major_ticks]

    fig.update_layout(
        xaxis=dict(
            range=x_range_log,
            type="log",
            tickvals=10**x_major_ticks,
            ticktext=x_tick_labels,
            title="Sample Duration (s)",
            ticks="outside",
            showline=True,
            mirror=True,
            title_standoff=0,
        ),
        yaxis=dict(
            range=y_range_log,
            type="log",
            tickvals=10**y_major_ticks,
            ticktext=y_tick_labels,
            ticks="outside",
            showline=True,
            mirror=True,
            title_standoff=4,
        ),
        font=dict(family="CMU Serif", size=18),
        legend=dict(
            x=0.4,
            y=0.8,
            xanchor="center",
            yanchor="middle",
            bgcolor="rgba(255,255,255,1.0)",
            font=dict(size=16),
            bordercolor="black",
            borderwidth=1,
        ),
        width=800,
        height=500,
        margin=dict(l=80, r=20, t=40, b=80),
        title=f"Multi-Phase: {key_to_title[key]}",
    )

    fig.show()

    # Create export directory
    EXPORT_DIR = "figures"
    os.makedirs(EXPORT_DIR, exist_ok=True)

    # Define filename stem
    filename_base = os.path.join(EXPORT_DIR, f"uncertainty_multiphase_{key}")

    # # Export to SVG and PDF
    # pio.write_image(fig, f"{filename_base}.pdf")
    # pio.write_image(fig, f"{filename_base}.png")

print("Multi-phase analysis complete!")

Running fits for phase φ = 0...
Starting parallel fits...


100%|██████████| 300/300 [00:25<00:00, 11.57it/s]

Running fits for phase φ = π/4...
Starting parallel fits...



100%|██████████| 300/300 [00:18<00:00, 16.65it/s]

Running fits for phase φ = π/2...
Starting parallel fits...


...

Running fits for phase φ = 3π/4...
Starting parallel fits...



100%|██████████| 300/300 [00:14<00:00, 20.02it/s]


Running fits for phase φ = π...
Starting parallel fits...


100%|██████████| 300/300 [00:23<00:00, 12.78it/s]


Multi-phase analysis complete!
